# Importing Libraries and Setup

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
import gradio as gr
from IPython.display import display, update_display, Markdown
import google.generativeai

In [2]:
import subprocess
import sys
import io

In [3]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, TextStreamer
from huggingface_hub import login, InferenceClient

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np

In [5]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

In [48]:
import random
import matplotlib
import requests

# Setting up API's 

In [7]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
claude_api_key = os.getenv('ANTHROPIC_API_KEY')
gemini_api_key = os.getenv('GOOGLE_API_KEY')
hf_token = os.getenv('HF_API_KEY')

In [8]:
login(hf_token, add_to_git_credential = True)

In [9]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

## Prompts and Models Setup

In [14]:
claude_model = 'claude-sonnet-4-5-20250929'
gpt_model = 'gpt-4o'
gemini_model = 'gemini-2.5-pro'

In [15]:
system_message = """
You are an advanced code-generation AI specialized in algorithmic trading systems.
Your goal is to generate clean, correct, and production-ready Python code that can:
1. Fetch historical market data using **free, no-API-key data sources**:
   - Yahoo Finance via the 'yfinance' Python library
   - Stooq via direct CSV URLs
2. Parse OHLCV price data and prepare it for analysis.
3. Implement algorithmic trading strategies.
4. Simulate trades and track:
   - Buy price
   - Sell price
   - Quantity
   - Profit/Loss (PnL)
   - Running equity curve
5. Generate simple visualizations:
   - Price chart
   - Buy/Sell markers
   - Optional equity curve
6. Return a list of Trade objects where:
   - Trade(ticker, quantity) represents a BUY (> 0) or SELL (< 0).
---------------------------------------
TRADING FUNCTION REQUIREMENTS
---------------------------------------
You will be shown one example trading function.
You must generate **five additional trading functions**:
- trade2()
- trade3()
- trade4()
- trade5()
- trade6()
Each function must:
- Implement a **unique** trading strategy
  (examples: momentum, mean reversion, moving average cross, RSI-based, random, risk-managed)
- Use real price data (from yfinance or Stooq)
- Include clear comments explaining the strategy logic
- Return a list of Trade objects
- Contain ONLY Python code (no explanations)
---------------------------------------
REALISTIC CONDITION THRESHOLDS
---------------------------------------
**IMPORTANT: Use realistic, achievable thresholds that trigger frequently in real market data.**
Do NOT use overly strict combinations. Examples of GOOD thresholds:
- Momentum strategies: Price change > 0.5% to 2% (not > 5%)
- Mean reversion: RSI < 40 OR RSI > 60 (not < 30 or > 70 exclusively)
- Moving average: Simple crossovers (MA5 > MA20) with no additional strict filters
- Volatility: Volatility > 1.5% to 2% (not > 3%)
- Combined conditions: Use OR logic, not strict AND logic
- Default to INCLUSIVE conditions that trigger on most trading days
Example BAD (too strict):
  if volatility > 0.03 and momentum > 0.02:  # Almost never triggers
Example GOOD (realistic):
  if momentum > 0.01 or (volatility > 0.015 and price_change > 0):  # Triggers regularly
---------------------------------------
DATA SOURCES ALLOWED
---------------------------------------
You must only use these free data sources:
1. **Yahoo Finance (recommended)**
   - Accessed via: import yfinance as yf
   - Does not require an API key
   - Provides historical OHLCV data
   - Suitable for backtesting and charting
2. **Stooq**
   - Direct CSV URL format:
     https://stooq.com/q/d/l/?s={TICKER}&i=d
   - No API key required
   - Good for daily OHLCV
No paid APIs, no API keys, no restricted services.
---------------------------------------
SIMULATION REQUIREMENTS
---------------------------------------
The generated Python code may need to simulate:
- Trade entries and exits
- PnL calculations
- Markers (buy/sell) for chart generation
- Optional equity curve
Keep simulation logic simple and clean.
---------------------------------------
OUTPUT RULES
---------------------------------------
- Your reply must contain ONLY Python code.
- No markdown, no explanation text, no commentary.
- Code must be executable as-is.
- Every function must return a list of Trade objects.
---------------------------------------
BEHAVIOR RULES
---------------------------------------
- Always generate safe, deterministic, bug-free code.
- Add comments inside the code explaining key logic.
- Avoid any external dependencies beyond:
   - yfinance
   - pandas
   - numpy
   - matplotlib
   - urllib / requests (optional for Stooq)
- Keep code readable and organized.
- Strategy variety is important—avoid repetition.
- Strategies MUST find signals in real market data regularly (not once per month).
You are a senior trading-systems engineer. Produce code at that standard.
"""

In [16]:
def user_prompt_for(trade_function):
    """
    Constructs the user prompt that will be sent to the model.
    It appends the provided example function, followed by detailed instructions
    consistent with the system prompt requirements.
    """
    return f"""
Below is an example trading function that demonstrates the required format,
structure, API usage, and return type:

{trade_function}

--------------------------------------
TASK FOR THE MODEL
--------------------------------------
Using the example function as a reference, write **five additional trading functions**:
- trade2()
- trade3()
- trade4()
- trade5()
- trade6()

Each function must:

1. Use only **free, no-API-key data sources**:
   - Yahoo Finance via `yfinance`
   - Stooq via direct CSV URLs

2. Fetch and process real historical OHLCV data for the tickers provided.

3. Implement **unique and distinct trading strategies**, such as:
   - Momentum / trend-following
   - Mean reversion
   - Moving average crossovers
   - RSI or indicator-based strategies
   - Volatility or risk-managed strategies
   - Randomized or Monte Carlo–style approaches

4. Include **clear comments inside the code** explaining:
   - The strategy logic  
   - Buy/sell conditions  
   - Any indicator calculations  
   - How decisions are made  

5. Return a list of `Trade` objects, where:
   - Trade(ticker, quantity) indicates BUY (> 0) or SELL (< 0).

6. Produce **ONLY Python code** in the output.
   - No markdown
   - No explanations
   - No additional text

Follow the structure of the example function and maintain clean, readable code.
"""

In [17]:
trading_function_example =  """
# tickers is a list of stock tickers (strings)
import tickers

# prices is a dict; the key is a ticker and the value is a list of historic prices, today first
import prices

# Trade represents a decision to buy or sell a quantity of a ticker
# Trade("IBM", 100) for a trade object representing purchasing 100 shares of IBM stock
# Trade("IBM", -50) for a trade object representing selling or shorting 50 shares of IBM stock
import Trade

import random
import numpy as np

def trade1():
    # Buy top performing stock in the last 5 days
    avg_prices = {ticker: np.mean(prices[ticker][:5]) for ticker in tickers}
    best_ticker = max(avg_prices, key=avg_prices.get)
    trade = Trade(best_ticker, 100)
    return [trade]
"""

In [18]:
def messages_for(trade_function):
    messages = [
        {'role':'system', 'content':system_message},
        {'role':'user','content':user_prompt_for(trade_function)}
    ]
    return messages

In [19]:
system_message_openai = """
You are an advanced code-generation AI specialized in algorithmic trading systems.
Your goal is to generate clean, correct, and production-ready Python code that can:
1. Fetch historical market data using **free, no-API-key data sources**:
   - Yahoo Finance via the 'yfinance' Python library
2. Parse OHLCV price data and prepare it for analysis.
3. Implement algorithmic trading strategies.
4. Return a list of Trade objects where:
   - Trade(ticker, quantity) represents a BUY (> 0) or SELL (< 0).

---------------------------------------
TRADING FUNCTION REQUIREMENTS
---------------------------------------
You will be shown one example trading function.
You must generate **five additional trading functions**:
- trade2()
- trade3()
- trade4()
- trade5()
- trade6()

Each function must:
- Implement a **unique** trading strategy
- Use real price data from yfinance
- Include clear comments explaining the strategy logic
- Return a list of Trade objects
- Contain ONLY Python code (no explanations)

---------------------------------------
CRITICAL: SIGNAL GENERATION REQUIREMENT
---------------------------------------
**IMPORTANT: Every strategy MUST generate trades for at least 5-15% of tickers.**

If your conditions are too strict, NO trades will be found.

ALWAYS include fallback conditions:
- Primary signal: Strict condition (crossover, breakover, etc.)
- Fallback signal: More inclusive condition that triggers regularly

Example (GOOD):
  if sma5 > sma20:  # Primary signal
      qty = 100
  elif sma5 > sma20 * 0.98:  # Fallback - close to crossover
      qty = 50
  elif price_change > 0.01:  # Extra fallback
      qty = 30

Example (BAD - will return 0 trades):
  if volatility > 0.03 and momentum > 0.02 and rsi < 40:  # Too strict AND logic

ALWAYS use OR logic for multiple conditions, not strict AND logic.

This ensures the strategy finds signals on most market days.

---------------------------------------
DATA SOURCES & REQUIREMENTS
---------------------------------------
1. Use yfinance to fetch data
2. Handle missing/None data gracefully
3. Use try-except blocks for downloads
4. Skip tickers that fail to download
5. Only use: yfinance, pandas, numpy, random, matplotlib

---------------------------------------
REALISTIC CONDITION THRESHOLDS
---------------------------------------
GOOD thresholds:
- Momentum: Price change > 0.5% to 2%
- Mean reversion: RSI < 40 OR RSI > 60
- Moving average: Simple crossovers (MA5 > MA20)
- Volatility: Volatility > 1.5% to 2%

Always use INCLUSIVE OR logic, not strict AND logic.

---------------------------------------
FALLBACK REQUIREMENT
---------------------------------------
**MANDATORY: Every function MUST have:**
1. Primary signal (strict condition)
2. Fallback signal (more inclusive)

This prevents returning 0 trades.

---------------------------------------
OUTPUT RULES
---------------------------------------
- Your reply must contain ONLY Python code.
- No markdown, no explanation text, no commentary.
- Code must be executable as-is.
- Every function must return a list of Trade objects.

---------------------------------------
BEHAVIOR RULES
---------------------------------------
- Always generate safe, deterministic, bug-free code.
- Add comments inside the code explaining key logic.
- Avoid external dependencies beyond: yfinance, pandas, numpy, random, matplotlib
- Keep code readable and organized.
- Strategy variety is important—avoid repetition.
- You are a senior trading-systems engineer. Produce code at that standard.
"""

In [20]:
def user_prompt_for_openai(trade_function):
    return f"""
Below is an example trading function:

{trade_function}

--------------------------------------
TASK FOR THE MODEL
--------------------------------------
Using the example function as a reference, write **five additional trading functions**:
- trade2()
- trade3()
- trade4()
- trade5()
- trade6()

REQUIREMENTS:
1. Use yfinance to fetch real historical OHLCV data
2. Implement **unique and distinct trading strategies**
3. Include **clear comments** explaining:
   - The strategy logic  
   - Buy/sell conditions  
   - Indicator calculations  
   - Threshold reasoning
4. Return a list of `Trade` objects
5. Produce **ONLY Python code** with no markdown or explanations

CRITICAL - FALLBACK CONDITIONS:
Each function MUST have:
- Primary signal (strict)
- Fallback signal (inclusive)

Example:
  if sma5 > sma20:
      qty = 100
  elif sma5 > sma20 * 0.98:
      qty = 50

This ensures trades are found regularly, not 0 every time.

Use OR logic, not AND logic. Use inclusive thresholds.
"""

In [21]:
def messages_for_openai(trade_function):
    messages = [
        {'role':'system', 'content':system_message_openai},
        {'role':'user','content':user_prompt_for_openai(trade_function)}
    ]
    return messages

# Frontier Models Streaming:

In [22]:
global_gpt_output = ''

def stream_gpt(trade_function):
    global global_gpt_output
    
    response = openai.chat.completions.create(
        model = gpt_model,
        messages = messages_for_openai(trade_function),
        stream = True
    )
    result = ''
    for chunk in response:
        result += chunk.choices[0].delta.content or ''
        result = result.replace('```','').replace('markdown','').replace("```python\n", "")
        yield result
        
    global_gpt_output = result # if += then it will add for every time new output generates

In [23]:
global_claude_output = ''

def stream_claude(trade_function):
    global global_claude_output
    
    response = claude.messages.stream(
        model = claude_model,
        messages = [{'role':'user','content':user_prompt_for(trade_function)}],
        system = system_message,
        max_tokens = 5000
    )
    result = ''
    with response as stream:
        for text in stream.text_stream:
            result += text
            result = result.replace('```','').replace('markdown','').replace("```python\n", "")
            yield result
    
    global_claude_output = result

In [24]:
global_gemini_output = ''

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

def stream_gemini(trade_function):
    global global_gemini_output
    
    gemini = google.generativeai.GenerativeModel(
        model_name = gemini_model, 
        system_instruction = system_message,
        safety_settings=safety_settings
    )
    response = gemini.generate_content(user_prompt_for(trade_function),stream = True)
    result = ''

    for chunk in response:
        if chunk.text:
            result += chunk.text or ''
            result = result.replace('```','').replace('markdown','').replace("```python\n", "")
            yield result

    global_gemini_output = result

# Open Sourced Model Streaming:

In [25]:
code_qwen = 'Qwen/CodeQwen1.5-7B-Chat'

code_gemma = 'google/codegemma-7b-it'
code_gemma_url = 'https://l77mnbo5de1yfudo.us-east-1.aws.endpoints.huggingface.cloud'

In [26]:
quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type = 'nf4'
)

In [27]:
global_qwen_output = ''

def stream_qwen(trade_function):
    global global_qwen_output
    
    messages = messages_for(trade_function)
    tokenizer = AutoTokenizer.from_pretrained(code_qwen, trust_remote_code = True)
    tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer.apply_chat_template(messages, return_tensors = 'pt', add_generation_prompt = True).to('mps')
    model = AutoModelForCausalLM.from_pretrained(
        code_qwen,
        quantization_config = quant_config,
        device_map="mps",        
        torch_dtype=torch.float16,
        use_cache=True
    )

    streamer = TextStreamer(tokenizer)
    output = model.generate(inputs, streamer = streamer, max_new_tokens = 5000)
    result = tokenizer.decode(output[0], skip_special_tokens = True)
    yield result

    global_qwen_output = result

In [28]:
global_gemma_output = ''

def stream_gemma(trade_function):
    global global_gemma_output 
    
    messages = [{'role':'user','content':user_prompt_for(trade_function)}]
    tokenizer = AutoTokenizer.from_pretrained(code_gemma, trust_remote_code= True)
    tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer.apply_chat_template(messages , tokenize = False, add_generation_prompt = True)

    client = InferenceClient(code_gemma_url, token = hf_token)
    stream = client.text_generation(inputs, stream = True, details = True, max_new_tokens = 3000)

    result = ''
    for chunk in stream:
        result += chunk.token.text or ''
        yield result

    global_gemma_output  = result

## Spliting for Trades

In [29]:
def split_trades(model,trade_function):
    splited = {}
    current = None
    if model == 'GPT':
        result = stream_gpt(trade_function)
    elif model == 'Claude':
        result = stream_claude(trade_function)
    elif model == 'Gemini':
        result = stream_gemini(trade_function)
    elif model == 'Qwen':
        result = stream_qwen(trade_function)
    elif model == 'Gemma':
        result = stream_gemma(trade_function)
    else:
        raise ValueError('Unknown Model')

    final_chunk= ''    
    for chunk in result:
        final_chunk += chunk
    
    for line in final_chunk.split("\n"):
        if line.strip().startswith('def trade'):
            clean = line.strip().split("(")[0].replace("def ","")
            if clean.endswith("python"):
                continue
            current = clean
            splited[current] = line + '\n'
        elif current:
            splited[current] += line + "\n"

    return splited

In [30]:
class Trade:
    def __init__(self, ticker, quantity):
        self.ticker = ticker
        self.quantity = quantity
        
    def __repr__(self): # how an object looks when printed.
        return f"Trade({self.ticker}, {self.quantity})"

## Claude Model

In [31]:
def execute_trade_nums_claude(model, trade_function, trade_num, tickers=['AAPL', 'MSFT', 'GOOGL']):
    code = split_trades(model, trade_function)
    trade_key = f"trade{trade_num}"
    
    if trade_key not in code:
        raise ValueError(f'{trade_key} not found. Available trades: {list(code.keys())}')
    
    try:
        output = io.StringIO()
        original_stdout = sys.stdout
        sys.stdout = output
        
        exec_globals = {
            **globals(),
            'tickers': tickers,
            'yf': yf,
            'pd': pd,
            'np': np,
            'Trade': Trade,
            'prices': prices,
        }
        
        exec(code[trade_key], exec_globals)
        trade_func = exec_globals[trade_key]
        result = trade_func()
        
    finally:
        sys.stdout = original_stdout
    
    return {
        'output': output.getvalue(),
        'result': result
    }

In [32]:
all_tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX', 
               'UBER', 'PYPL', 'AMD', 'INTC', 'COIN', 'RIOT', 'MARA', 'GME', 'AMC',
               'F', 'GE', 'T', 'IBM', 'CSCO', 'DELL', 'HPQ', 'ORCL', 'CRM', 'ADBE', 
               'SPLK', 'SNOW', 'DDOG', 'MDB', 'CRWD', 'ZM', 'ROKU', 'PINS', 'SNAP', 
               'TWTR', 'RBLX', 'U', 'NET', 'OKTA', 'FEYE', 'FTNT', 'PANW', 'WDAY', 'VRSN']

prices = {}
for ticker in all_tickers:
    try:
        # Download data for single ticker (not multiple)
        data = yf.download(ticker, period='60d', progress=False)
        if not data.empty:
            # Access Close directly (it's a Series for single ticker)
            # Flatten any nested structure
            close_prices = data['Close'].values.flatten().tolist()
            prices[ticker] = close_prices
    except Exception as e:
        print(f"Failed to download {ticker}: {e}")

print(f"Successfully fetched prices for {len(prices)} tickers")

if prices:
    ticker = list(prices.keys())[0]
    print(f"\nVerification - {ticker} first 5 prices:", prices[ticker][:5])
    print(f"Type of first value:", type(prices[ticker][0]))


1 Failed download:
['SPLK']: YFPricesMissingError('possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['TWTR']: YFPricesMissingError('possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['FEYE']: YFPricesMissingError('possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")')


Successfully fetched prices for 43 tickers

Verification - AAPL first 5 prices: [231.91517639160156, 229.49752807617188, 238.2390594482422, 239.5477752685547, 239.45787048339844]
Type of first value: <class 'float'>


In [55]:
def execute_all_trades_claude(model, trade_function, trade_num):
    standard_tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX', 'UBER', 'PYPL', 'AMD', 'INTC', 'COIN', 'RIOT', 'MARA', 'GME', 'AMC']
    extended_tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX', 'UBER', 'PYPL', 'AMD', 'INTC', 'COIN', 'RIOT', 'MARA', 'GME', 'AMC', 'F', 'GE', 'T', 'IBM', 'CSCO', 'DELL', 'HPQ', 'ORCL', 'CRM', 'ADBE', 'SPLK', 'SNOW', 'DDOG', 'MDB', 'CRWD', 'ZM', 'ROKU', 'PINS', 'SNAP', 'TWTR', 'RBLX', 'U', 'NET', 'OKTA', 'FEYE', 'FTNT', 'PANW', 'WDAY', 'VRSN']
    
    tickers = extended_tickers if trade_num == 3 else standard_tickers
    # trade3 is for Volatility breakout and stuff.

    available_tickers = [t for t in tickers if t in prices]
    #print(f"Using {len(available_tickers)} out of {len(tickers)} tickers")
    
    result = execute_trade_nums_claude(model, trade_function, trade_num, tickers=available_tickers)
    trades = result['result']
    
    output = f"Trade {trade_num}: {len(trades)} trades\n\n"
    output += f"{'Ticker':<10} {'Quantity':<10} {'Action':<10}\n"
    output += "-" * 35 + "\n"
    
    for trade in trades:
        action = "BUY" if trade.quantity > 0 else "SELL"
        output += f"{trade.ticker:<10} {trade.quantity:<10} {action:<10}\n"
    
    return output

## Gemini Model

In [56]:
def execute_trade_nums_gemini(trade_function, trade_num, tickers=['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX', 'UBER', 'PYPL', 'AMD', 'INTC', 'COIN', 'RIOT', 'MARA', 'GME', 'AMC', 'F', 'GE', 'T', 'IBM', 'CSCO', 'DELL', 'HPQ', 'ORCL', 'CRM', 'ADBE', 'SPLK', 'SNOW', 'DDOG', 'MDB', 'CRWD', 'ZM', 'ROKU', 'PINS', 'SNAP', 'TWTR', 'RBLX', 'U', 'NET', 'OKTA', 'FEYE', 'FTNT', 'PANW', 'WDAY', 'VRSN']):
    code = split_trades('Gemini',trade_function)
    trade_key = f"trade{trade_num}"

    try:
        output = io.StringIO()
        original_stdout = sys.stdout
        sys.stdout = output

        exec_globals = {
            **globals(),
            'tickers': tickers,
            'TICKERS': tickers,
            'TICKER_LIST': tickers,
            'yf': yf,
            'pd': pd,
            'np': np,
            'Trade': Trade,
            'random': random, 
            'matplotlib': matplotlib,
            'requests': requests,
            'prices': prices
        }

        exec(code[trade_key], exec_globals)
        trade_func = exec_globals[trade_key]
        result = trade_func()

    except ValueError as e:
        if "ambiguous" in str(e):
            result = []  # Return empty if pandas ambiguity error
        elif "identically-labeled" in str(e):
            result = []
        else:
            raise
            
    finally:
        sys.stdout = original_stdout
        warnings.filterwarnings('default')
        
    return {
        'output': output.getvalue(),
        'result': [
            {'ticker': t.ticker, 'quantity': t.quantity, 'action': 'BUY' if t.quantity > 0 else 'SELL'}
            for t in result
        ]
    } # t is loop variable. 't' is One Trade basically

In [57]:
def execute_with_validation_gemini(trade_function,trade_num, max_retries=10):
    for attempt in range(max_retries):
        result = execute_trade_nums_gemini(trade_function, trade_num, tickers=['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX', 'UBER', 'PYPL', 'AMD', 'INTC', 'COIN', 'RIOT', 'MARA', 'GME', 'AMC', 'F', 'GE', 'T', 'IBM', 'CSCO', 'DELL', 'HPQ', 'ORCL', 'CRM', 'ADBE', 'SPLK', 'SNOW', 'DDOG', 'MDB', 'CRWD', 'ZM', 'ROKU', 'PINS', 'SNAP', 'TWTR', 'RBLX', 'U', 'NET', 'OKTA', 'FEYE', 'FTNT', 'PANW', 'WDAY', 'VRSN'])
        if result['result']: 
            print(f"✓ Trade {trade_num}: Found {len(result['result'])} trades on attempt {attempt+1}")
            return result
        print(f"✗ Trade {trade_num}: Attempt {attempt+1} - No trades found, retrying...")
    print(f"✗ Trade {trade_num}: Failed after {max_retries} attempts")
    print(f"⚠️  Strategy is too strict. Try a different trade number instead.")
    print(f"💡 Suggestion: Use trade4 or trade6 (usually more reliable)\n")
    return result  

## OpenAI Model

In [58]:
def execute_trade_nums_openai(trade_function, trade_num, tickers=['AAPL', 'MSFT', 'GOOGL']):
    
    code = split_trades('GPT', trade_function)
    trade_key = f"trade{trade_num}"
    
    def _as_list_of_symbols():
        return tickers
    
    def _get_ohlcv(sym, source_preference="yf", min_rows=30):
        try:
            df = yf.download(sym, period="60d", progress=False)
            if len(df) < min_rows:
                return None
            return df
        except:
            return None
    
    def _sma(series, window):
        return series.rolling(window=window).mean()
    
    try:
        output = io.StringIO()
        original_stdout = sys.stdout
        sys.stdout = output
        
        exec_globals = {
            **globals(),
            'tickers': tickers,
            'TICKERS': tickers,
            'TICKER_LIST': tickers,
            'yf': yf,
            'pd': pd,
            'np': np,
            'Trade': Trade,
            'random': random, 
            'matplotlib': matplotlib,
            'requests': requests,
            '_as_list_of_symbols': _as_list_of_symbols,
            '_get_ohlcv': _get_ohlcv,
            '_sma': _sma
        }
        
        exec(code[trade_key], exec_globals)
        trade_func = exec_globals[trade_key]
        
        try:
            result = trade_func()
        except TypeError:
            result = trade_func(tickers) #  [> 0] positional arguments
    
    except Exception as e:
        result = []
    
    finally:
        sys.stdout = original_stdout
        warnings.filterwarnings('default')
    
    return {
        'output': output.getvalue(),
        'result': [
            {'ticker': t.ticker, 'quantity': t.quantity, 'action': 'BUY' if t.quantity > 0 else 'SELL'}
            for t in result
        ]
    }

### GPT's startegies were very inconsistent, every time they keep on changing giving empty trades. So, I add retries.

In [59]:
def execute_with_validation_openai(trade_function,trade_num, max_retries=10):
    for attempt in range(max_retries):
        result = execute_trade_nums_openai(trade_function, trade_num, tickers=['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX', 'UBER', 'PYPL', 'AMD', 'INTC', 'COIN', 'RIOT', 'MARA', 'GME', 'AMC', 'F', 'GE', 'T', 'IBM', 'CSCO', 'DELL', 'HPQ', 'ORCL', 'CRM', 'ADBE', 'SPLK', 'SNOW', 'DDOG', 'MDB', 'CRWD', 'ZM', 'ROKU', 'PINS', 'SNAP', 'TWTR', 'RBLX', 'U', 'NET', 'OKTA', 'FEYE', 'FTNT', 'PANW', 'WDAY', 'VRSN'])
        if result['result']: 
            print(f"✓ Trade {trade_num}: Found {len(result['result'])} trades on attempt {attempt+1}")
            return result
        print(f"✗ Trade {trade_num}: Attempt {attempt+1} - No trades found, retrying...")
    print(f"✗ Trade {trade_num}: Failed after {max_retries} attempts")
    print(f"⚠️  Strategy is too strict. Try a different trade number instead.")
    print(f"💡 Suggestion: Use trade4 or trade6 (usually more reliable)\n")
    return result  

## Trade Strategies:

In [60]:
def optimize(model, trade_function):
    if model == 'GPT':
        result = stream_gpt(trade_function)
    elif model == 'Claude':
        result = stream_claude(trade_function)
    elif model == 'Gemini':
        result = stream_gemini(trade_function)
    elif model == 'Qwen':
        result = stream_qwen(trade_function)
    elif model == 'Gemma':
        result = stream_gemma(trade_function)
    else:
        raise ValueError('Unknown Model')

    for chunk in result:
        yield chunk

## Splitted Trade Results

In [61]:
def display_splitted_trade(generated_code, model, trade_function, trade_num):
    if generated_code and generated_code.strip() != '':
        print("✓ Using already generated code...")
        code = {}
        current = None
        
        for line in generated_code.split("\n"):
            if line.strip().startswith('def trade'):
                clean = line.strip().split("(")[0].replace("def ", "")
                if clean.endswith("python"):
                    continue
                current = clean
                code[current] = line + '\n'
            elif current:
                code[current] += line + "\n"
    else:
        print("No generated code. Regenerating from model...")
        code = split_trades(model, trade_function)
    
    trade_key = f"trade{int(trade_num)}"
    if trade_key in code:
        return code[trade_key]
    else:
        return f"# {trade_key} not found"

## Sanitize the Results (Gemini Trade 3 Error) - 

In [68]:
def sanitize_generated_code(code: str) -> str:
    """
    Fix common Pandas errors - simple string replacement, no regex
    """
    lines = code.split('\n')
    result_lines = []
    
    for line in lines:
        # Skip lines that already have .any() or .all()
        if '.any()' in line or '.all()' in line:
            result_lines.append(line)
            continue
        
        # Fix: Convert last_row['X'] to last_row['X'].values[0]
        if "last_row['" in line and any(op in line for op in ['>', '<', '==', '!=']):
            # Find all occurrences of last_row['something']
            while "last_row['" in line and ".values[0]" not in line:
                # Find the pattern: last_row['Close']
                start = line.find("last_row['")
                if start == -1:
                    break
                end = line.find("']", start) + 2  # Include the ']
                old_text = line[start:end]
                new_text = old_text + ".values[0]"
                line = line.replace(old_text, new_text, 1)  # Replace only first occurrence
        
        # Fix: Look for if statements with comparisons
        if 'if ' in line and any(op in line for op in ['>', '<', '==', '!=']):
            indent = len(line) - len(line.lstrip())
            if_part = line[indent:].replace('if ', '', 1)
            
            if if_part.endswith(':'):
                condition = if_part[:-1].strip()
                
                # Skip simple comparisons
                if any(skip in condition for skip in ['len(', '==True', '==False', '!=True', '!=False', 'in ', 'is ', '.values[0]']):
                    result_lines.append(line)
                    continue
                
                # Add .any() to the condition
                new_line = ' ' * indent + f'if ({condition}).any():'
                result_lines.append(new_line)
                continue
        
        # Fix: DataFrame with scalar values - add index parameter
        if 'pd.DataFrame(' in line and 'index=' not in line:
            line = line.replace('pd.DataFrame(', 'pd.DataFrame(index=[0], ')
        
        result_lines.append(line)
    
    return '\n'.join(result_lines)

## Display the Result

In [69]:
def optimize_value(splitted_code, model, trade_function, trade_num, max_retries=10):
    
    print(f"DEBUG START: splitted_code length = {len(splitted_code) if splitted_code else 0}")
    print(f"DEBUG: model = {model}, trade_num = {trade_num}")
    
    if splitted_code and splitted_code.strip() and splitted_code.strip() != "":
        print("✓ Using already displayed code from splitted_code...")

        standard_tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX', 
                            'UBER', 'PYPL', 'AMD', 'INTC', 'COIN', 'RIOT', 'MARA', 'GME', 'AMC']
        extended_tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX', 
                            'UBER', 'PYPL', 'AMD', 'INTC', 'COIN', 'RIOT', 'MARA', 'GME', 'AMC', 
                            'F', 'GE', 'T', 'IBM', 'CSCO', 'DELL', 'HPQ', 'ORCL', 'CRM', 'ADBE', 
                            'SPLK', 'SNOW', 'DDOG', 'MDB', 'CRWD', 'ZM', 'ROKU', 'PINS', 'SNAP', 
                            'TWTR', 'RBLX', 'U', 'NET', 'OKTA', 'FEYE', 'FTNT', 'PANW', 'WDAY', 'VRSN']

        tickers = extended_tickers if trade_num == 3 else standard_tickers
        available_tickers = [t for t in tickers if t in prices]
        trade_key = f"trade{int(trade_num)}"
        
        print(f"DEBUG: Available tickers: {len(available_tickers)}")
        print(f"DEBUG: Trade key: {trade_key}")

        def download_data(ticker, period="60d"):  
            try:
                return yf.download(ticker, period=period, progress=False)
            except:
                return None
        
        def _as_list_of_symbols():
            return tickers
        
        def _get_ohlcv(sym, source_preference="yf", min_rows=30):
            try:
                df = yf.download(sym, period="60d", progress=False)
                if len(df) < min_rows:
                    return None
                return df
            except:
                return None
        
        def _sma(series, window):
            return series.rolling(window=window).mean()
            
        try:
            output = io.StringIO()
            original_stdout = sys.stdout
            sys.stdout = output
            
            exec_globals = {
                **globals(),
                'tickers': tickers,
                'TICKERS': tickers,
                'TICKER_LIST': tickers,
                'yf': yf,
                'pd': pd,
                'np': np,
                'Trade': Trade,
                'prices': prices,
                'random': random, 
                'matplotlib': matplotlib,
                'requests': requests,
                '_as_list_of_symbols': _as_list_of_symbols,
                '_get_ohlcv': _get_ohlcv,
                '_sma': _sma,
                'download_data': download_data,
                'TRADE_QUANTITY': 100,
            }

            print(f"DEBUG: About to exec splitted_code")
            sanitized_code = sanitize_generated_code(splitted_code)
            exec(sanitized_code, exec_globals)
            print(f"DEBUG: exec completed")
            
            trade_func = exec_globals[trade_key]
            print(f"DEBUG: trade_func found: {trade_func}")
            
            result = trade_func()
            print(f"DEBUG: Result from trade_func: {result}, type: {type(result)}, len: {len(result)}")

        except ValueError as e:
            print(f"DEBUG: ValueError caught: {e}")
            if "ambiguous" in str(e):
                result = []
            elif "identically-labeled" in str(e):
                result = []
            else:
                raise

            try:
                result = trade_func()
            except TypeError:
                result = trade_func(tickers)
        
        except (ValueError, KeyError, TypeError) as e:
            result = []
        except Exception as e:
            result = []
        
        finally:
            sys.stdout = original_stdout
        
        print(f"DEBUG: Final result length: {len(result)}")
        
        output_text = f"Trade {trade_num}: {len(result)} trades\n\n"
        output_text += f"{'Ticker':<10} {'Quantity':<10} {'Action':<10}\n"
        output_text += "-" * 35 + "\n"
        
        for trade in result:
            action = "BUY" if trade.quantity > 0 else "SELL"
            output_text += f"{trade.ticker:<10} {trade.quantity:<10} {action:<10}\n"
        
        return output_text

    else:
        print("✓ No code in splitted_code, using original execution method...")
        if model == "GPT":
            result = execute_with_validation_openai(trade_function, trade_num, max_retries=10)
        elif model == "Claude":
            result = execute_all_trades_claude(model, trade_function, trade_num)
        elif model == "Gemini":
            result = execute_with_validation_gemini(trade_function, trade_num, max_retries=10)
        else:
            raise ValueError("Please try only with GPT, Claude, Gemini")
        
        return result


# Gradio UI

In [70]:
with gr.Blocks() as ui:
    gr.Markdown('## Trading Advice Simulator')
    with gr.Row():
        trade1_input = gr.Textbox(label ='Trade 1', lines = 20, value =trading_function_example)
        output_trade = gr.Textbox(label = 'Generated Trading Decisions', lines = 20)
    with gr.Row():
        model = gr.Dropdown(['GPT','Claude','Gemini','Qwen','Gemma'],label = 'Select Model')
    with gr.Row():
        generate_btn = gr.Button('Simulate Trading')
    with gr.Row():
        splitted_code = gr.TextArea(label = 'Generated Trade Strategy Code')
        value = gr.TextArea(label = 'Trade Results')
    with gr.Row():
        slider_inputs=gr.Slider(2, 6, step=1, label="Trade Number",info="Choose which trading strategy to run")
    with gr.Row():
        split_btn = gr.Button('View Selected Trade Code')
        trade_btn = gr.Button('Run Trading Strategy')

    generate_btn.click(fn=optimize,inputs=[model, trade1_input], outputs=[output_trade])
    split_btn.click(fn = display_splitted_trade, inputs = [output_trade, model, trade1_input, slider_inputs], outputs = [splitted_code])
    trade_btn.click(fn = optimize_value, inputs = [splitted_code,model,trade1_input,slider_inputs], outputs = [value])
ui.launch()

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=150, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=157, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


No generated code. Regenerating from model...
DEBUG START: splitted_code length = 0
DEBUG: model = Gemini, trade_num = 4
✓ No code in splitted_code, using original execution method...


<string>:23: FutureWarning: YF.download() has changed argument auto_adjust default to True


✗ Trade 4: Attempt 1 - No trades found, retrying...


...

DEBUG: Final result length: 0
No generated code. Regenerating from model...
DEBUG START: splitted_code length = 1583
DEBUG: model = Gemini, trade_num = 3
✓ Using already displayed code from splitted_code...
DEBUG: Available tickers: 43
DEBUG: Trade key: trade3


<string>:18: FutureWarning: YF.download() has changed argument auto_adjust default to True


DEBUG: Final result length: 0
No generated code. Regenerating from model...
DEBUG START: splitted_code length = 1636
DEBUG: model = Gemini, trade_num = 2
✓ Using already displayed code from splitted_code...
DEBUG: Available tickers: 17
DEBUG: Trade key: trade2


<string>:14: FutureWarning: YF.download() has changed argument auto_adjust default to True


DEBUG: Final result length: 0
No generated code. Regenerating from model...
DEBUG START: splitted_code length = 1050
DEBUG: model = Gemini, trade_num = 5
✓ Using already displayed code from splitted_code...
DEBUG: Available tickers: 17
DEBUG: Trade key: trade5
DEBUG: Final result length: 0
No generated code. Regenerating from model...
DEBUG START: splitted_code length = 825
DEBUG: model = Gemini, trade_num = 6
✓ Using already displayed code from splitted_code...
DEBUG: Available tickers: 17
DEBUG: Trade key: trade6
DEBUG: Final result length: 1
DEBUG START: splitted_code length = 825
DEBUG: model = Gemini, trade_num = 6
✓ Using already displayed code from splitted_code...
DEBUG: Available tickers: 17
DEBUG: Trade key: trade6
DEBUG: Final result length: 1
No generated code. Regenerating from model...
DEBUG START: splitted_code length = 2128
DEBUG: model = Claude, trade_num = 6
✓ Using already displayed code from splitted_code...
DEBUG: Available tickers: 17
DEBUG: Trade key: trade6
DEBU